In [ ]:
# ===========================================================
# Connect ArcGIS
# ===========================================================
import os, json, time, pathlib, traceback, threading, requests
from urllib.parse import parse_qsl
from arcgis.gis import GIS

import ipywidgets as W
from IPython.display import display, HTML

# ---------------------------- CSS / Presentation ----------------------------
display(HTML(r"""
<style>
  :root{
    /* fresh yellow theme */
    --brandA:#f59e0b; --brandB:#fde047; /* amber-500 → yellow-300 */
    --ink:#0f172a; --ok:#10b981; --err:#ef4444; --muted:#6b7280;
    --card:#ffffff; --panel:#fafafa; --border:#e5e7eb;
  }
  .edc-shell{font-family: Inter, Segoe UI, Roboto, system-ui, -apple-system, Arial; max-width:1024px; margin:0 auto;}
  .edc-header{
    background: linear-gradient(90deg, var(--brandA), var(--brandB));
    color:#0b0f19; padding:22px 20px; border-radius:16px; margin:16px 0 10px;
    box-shadow: 0 8px 28px rgba(0,0,0,.10); text-align:center;
  }
  .edc-header h1{ margin:0; font-weight:900; font-size:28px; letter-spacing:.2px; }
  .edc-sub{ color:#3f3f46; margin-top:4px; }

  .edc-required-strip{ margin:4px 0 8px; text-align:left; }
  .edc-required-chip{
    display:inline-block; background:#fff7ed; color:#9a3412;
    border-radius:999px; padding:3px 10px; font-size:11px; font-weight:800;
  }

  .edc-grid{ display:grid; grid-template-columns: 1fr; gap:12px; }
  @media (min-width:900px){ .edc-grid{ grid-template-columns: 1fr 1fr; } }

  .edc-card{
    background:var(--card); border:1px solid var(--border); border-radius:14px;
    padding:16px; margin:12px 0; box-shadow: 0 2px 12px rgba(0,0,0,.06);
  }
  .edc-title{ font-weight:900; color:var(--ink); margin:0 0 8px; font-size:16px; }
  .edc-muted{ color:var(--muted); font-size:12px; margin-top:6px; }

  .edc-row{ display:flex; gap:12px; flex-wrap:wrap; align-items:center; }
  .edc-row > * { flex: 1 1 300px; }

  .edc-buttons .widget-button{ margin-right:8px; }
  .edc-log{
    border:1px solid var(--border); border-radius:12px; padding:12px; height:320px; overflow:auto;
    background:var(--panel); font-family: ui-monospace, SFMono-Regular, Menlo, Consolas, monospace; font-size:12px;
  }
  .edc-msg-ok{ background:#ecfdf5; border:1px solid #a7f3d0; color:#065f46; padding:10px 12px; border-radius:10px; margin-top:8px; }
  .edc-msg-err{ background:#fef2f2; border:1px solid #fecaca; color:#991b1b; padding:10px 12px; border-radius:10px; margin-top:8px; font-weight:600; }
</style>
<div class="edc-shell">
  <div class="edc-header">
    <h1>Connect ArcGIS</h1>
    <div class="edc-sub"></div>
  </div>
  <div class="edc-required-strip">
    <span class="edc-required-chip">Required fields are marked with *</span>
  </div>
</div>
"""))

# ---------------------------- Runtime state ----------------------------
stop_event = threading.Event()
active_transfer_id = {"id": None}

# ---------------------------- Helpers ----------------------------
def _hdr(api_key=None, bearer=None):
    h = {"Accept":"application/json"}
    if api_key: h["X-Api-Key"] = api_key
    if bearer:  h["Authorization"] = f"Bearer {bearer}"
    return h

def _parse_qs(qs):
    if not qs: return None
    return {k:v for k,v in parse_qsl(qs, keep_blank_values=True)}

def _guess_arcgis_type(path):
    p = pathlib.Path(path)
    if p.suffix.lower() in [".geojson",".json"]:
        try:
            with open(path,"r",encoding="utf-8") as f:
                j=json.load(f)
            if isinstance(j,dict) and j.get("type") in ("FeatureCollection","Feature","GeometryCollection"):
                return "GeoJson"
        except Exception:
            pass
        return "JSON"
    if p.suffix.lower()==".csv": return "CSV"
    if p.suffix.lower() in [".shp",".zip"]: return "Shapefile"
    return "File"

# ---------------------------- EDC Core ----------------------------
class Cancelled(RuntimeError): pass

def start_transfer(mgmt_url, provider_dsp, api_key, bearer, agreement_id,
                   transfer_type="HttpData-PULL", asset_id=None,
                   http_method=None, http_path=None, http_query=None, http_body=None,
                   push_base_url=None, connector_id=None,
                   verify_ssl=True, timeout=120, log_out=None):
    """
    Sends a TransferRequest from the CONSUMER (this mgmt_url) to the PROVIDER (provider_dsp).
    Pull: proxies HTTP and returns EDR later; Push: provider pushes to your baseUrl.
    """
    req = {
        "@context": {"@vocab": "https://w3id.org/edc/v0.0.1/ns/"},
        "@type": "TransferRequest",
        "protocol": "dataspace-protocol-http",
        "counterPartyAddress": provider_dsp,
        "contractId": agreement_id,
        "transferType": transfer_type
    }
    if connector_id:  
        req["connectorId"] = connector_id

    if transfer_type == "HttpData-PULL":
        if asset_id: req["assetId"] = asset_id
        req["dataDestination"] = {"@type": "DataAddress", "type": "HttpProxy"}
        props = {}
        if http_method: props["method"] = http_method
        if http_path:   props["path"] = http_path
        qd = _parse_qs(http_query)
        if qd:          props["queryParams"] = qd
        if http_body:   props["body"] = http_body
        if props:       req["properties"] = props
    else:  # PUSH
        if not push_base_url:
            raise ValueError("PUSH flow requires a sink baseUrl.")
        if asset_id: req["assetId"] = asset_id
        req["dataDestination"] = {"@type": "DataAddress", "type": "HttpData", "baseUrl": push_base_url}

    url = f"{mgmt_url.rstrip('/')}/v3/transferprocesses"
    if log_out:
        print("POST", url); print(json.dumps(req, indent=2))

    r = requests.post(url, headers={**_hdr(api_key,bearer), "Content-Type": "application/json"},
                      data=json.dumps(req), timeout=timeout, verify=verify_ssl)
    if log_out:
        print("→ HTTP", r.status_code)
        print(r.text[:1200] + ("..." if len(r.text) > 1200 else ""))
    r.raise_for_status()

    # extract id across variants
    try: j = r.json()
    except Exception: j = {}
    tp_id = (j.get("id") or j.get("@id") or j.get("transferProcessId") or j.get("value"))
    if not tp_id:
        loc = r.headers.get("Location") or r.headers.get("location")
        if loc: tp_id = loc.rstrip("/").split("/")[-1]
    if not tp_id:
        raise RuntimeError(f"No transfer id in response: {j}")

    if log_out: print("transferProcessId:", tp_id)
    active_transfer_id["id"] = tp_id
    return tp_id

def try_terminate_transfer(mgmt_url, tp_id, api_key, bearer, verify_ssl=True, timeout=60, log_out=None):
    """
    Best-effort terminate (not all distros support). Tries common endpoints.
    """
    if not tp_id: return False
    h = _hdr(api_key, bearer)
    candidates = [
        f"{mgmt_url.rstrip('/')}/v3/transferprocesses/{tp_id}/terminate",
        f"{mgmt_url.rstrip('/')}/v3/transferprocesses/{tp_id}/deprovision",
        f"{mgmt_url.rstrip('/')}/v3/transferprocesses/{tp_id}/cancel",
    ]
    for u in candidates:
        try:
            if log_out: print("POST", u)
            rr = requests.post(u, headers=h, timeout=timeout, verify=verify_ssl)
            if rr.status_code in (200, 202, 204):
                if log_out: print("→ terminate accepted")
                return True
        except Exception:
            pass
    return False

def poll_until_completed(mgmt_url, tp_id, api_key, bearer,
                         verify_ssl=True, timeout=120, max_polls=120, poll_s=2, log_out=None):
    h = _hdr(api_key,bearer)
    url = f"{mgmt_url.rstrip('/')}/v3/transferprocesses/{tp_id}"
    last = None
    for i in range(max_polls):
        if stop_event.is_set():
            raise Cancelled("User pressed Stop.")
        r = requests.get(url, headers=h, timeout=timeout, verify=verify_ssl)
        r.raise_for_status()
        s = r.json(); last = s
        st = s.get("state")
        if log_out: print(f"Poll {i+1}/{max_polls} → {st}")
        if st == "COMPLETED":
            return s
        if st in ("ERROR","TERMINATED"):
            detail = s.get("failureDetail") or s.get("errorDetail") or s.get("reason") or s
            raise RuntimeError(f"Transfer failed: {json.dumps(detail, indent=2)}")
        time.sleep(poll_s)
    raise TimeoutError(f"Transfer did not complete in time. Last state: {json.dumps(last, indent=2)}")

def fetch_edr(mgmt_url, tp_id, api_key, bearer, verify_ssl=True, timeout=120, retries=8, wait_s=1.5, log_out=None):
    h = _hdr(api_key,bearer)
    candidates = [
        f"{mgmt_url.rstrip('/')}/v3/transferprocesses/{tp_id}/edr",
        f"{mgmt_url.rstrip('/')}/v3/edrs/{tp_id}",
        f"{mgmt_url.rstrip('/')}/v3/edrs?transferProcessId={tp_id}",
    ]
    for i in range(1, retries+1):
        if stop_event.is_set():
            raise Cancelled("User pressed Stop.")
        for url in candidates:
            if log_out: print("GET", url)
            r = requests.get(url, headers=h, timeout=timeout, verify=verify_ssl)
            if r.status_code == 200 and r.text.strip():
                try: data = r.json()
                except Exception: data = None
                if isinstance(data, list) and data: data = data[0]
                if isinstance(data, dict):
                    endpoint = data.get("endpoint") or data.get("endpointUrl")
                    auth_key = data.get("authKey") or data.get("authorizationHeader") or "Authorization"
                    auth_val = data.get("authCode") or data.get("authValue") or data.get("authorizationToken")
                    if endpoint and auth_val:
                        if log_out: print("EDR:", json.dumps({"endpoint": endpoint, "authKey": auth_key}, indent=2))
                        return {"endpoint": endpoint, "authKey": auth_key, "authVal": auth_val}
        if log_out: print(f"EDR not ready, retry {i}/{retries} …")
        time.sleep(wait_s)
    raise RuntimeError("No EDR available after retries.")

def download_via_edr(edr, out_path, verify_ssl=True, timeout=120, retries=4, wait_s=1.5, log_out=None):
    endpoint, auth_key, auth_val = edr["endpoint"], edr["authKey"], edr["authVal"]
    hdr = {"Accept": "*/*", auth_key: auth_val}
    last_err = None
    for i in range(1, retries+1):
        if stop_event.is_set():
            raise Cancelled("User pressed Stop.")
        if log_out: print("GET (EDR)", endpoint)
        try:
            r = requests.get(endpoint, headers=hdr, timeout=timeout, verify=verify_ssl)
            if r.status_code >= 400:
                if log_out: print(r.text[:400])
                r.raise_for_status()
            content = r.content
            if not content:
                raise RuntimeError("EDR download returned empty body.")
            pathlib.Path(out_path).parent.mkdir(parents=True, exist_ok=True)
            with open(out_path, "wb") as f:
                f.write(content)
            if log_out: print(f"Saved {len(content)} bytes → {out_path}")
            return out_path
        except Exception as e:
            last_err = e
            if log_out: print(f"EDR download retry {i}/{retries} failed: {e}")
            time.sleep(wait_s)
    raise last_err or RuntimeError("EDR download failed.")

# ---------------------------- ArcGIS publish ----------------------------
def publish_to_arcgis(file_path, title, tags, desc, auth_mode, portal_url, portal_user, portal_pass, log_out=None):
    gis = "pro" if auth_mode == "pro" else (portal_url, portal_user, portal_pass)
    gis = GIS(gis) if isinstance(gis, str) else GIS(*gis)
    arc_type = _guess_arcgis_type(file_path)
    props = {"title": title, "type": arc_type, "tags": tags, "description": desc}
    item = gis.content.add(item_properties=props, data=file_path)
    if log_out: print(f"Added item: {getattr(item,'id',item)}")
    if arc_type in ("GeoJson", "CSV", "Shapefile"):
        try:
            published = item.publish()
            if log_out: print(f"Published hosted feature layer: {published.id}")
            return published
        except Exception as ex:
            if log_out: print("Publish skipped:", ex)
            return item
    return item

# ---------------------------- UI Controls ----------------------------
# Mode toggle
mode = W.ToggleButtons(
    options=[("Pull Transfer","pull"),("Push Transfer","push")],
    value="pull", description="Mode *"
)

# Common connector/auth
mgmt_url   = W.Text(value="https://consumer-connector.example.com/api/management",
                    description="Mgmt API *", layout=W.Layout(width="720px"))
dsp_url    = W.Text(value="https://provider-connector.example.com/api/dsp",
                    description="Provider DSP *", layout=W.Layout(width="720px"))
connector_id = W.Text(value="", description="Connector ID", layout=W.Layout(width="350px"),
                      placeholder="optional hint for some runtimes")
agreement  = W.Text(value="", description="Contract ID *", layout=W.Layout(width="720px"))
api_key    = W.Password(value="", description="API Key", layout=W.Layout(width="350px"))
bearer_tok = W.Password(value="", description="Bearer", layout=W.Layout(width="350px"))

# Pull tab
asset_id   = W.Text(value="", description="Asset ID", layout=W.Layout(width="350px"),
                    placeholder="Optional if bound by contract")
http_method= W.Dropdown(options=["GET","POST","PUT","DELETE"], value="GET", description="HTTP", layout=W.Layout(width="140px"))
http_path  = W.Text(value="", description="Path", layout=W.Layout(width="720px"), placeholder="/api/data")
http_query = W.Text(value="", description="Query", layout=W.Layout(width="720px"), placeholder="a=1&b=2")
http_body  = W.Textarea(value="", description="Body", layout=W.Layout(width="720px", height="64px"))

# Push tab
push_base_url = W.Text(value="", description="Sink baseUrl *", layout=W.Layout(width="720px"),
                       placeholder="https://your-sink.example.com/webhook")
push_note = W.HTML('<div class="edc-muted">For PUSH, provider will deliver to your URL; this UI doesn’t download.</div>')

# Output & ArcGIS
out_file   = W.Text(value=r"C:\Temp\connector_payload.geojson", description="Save to *", layout=W.Layout(width="720px"))
title      = W.Text(value="Connector → ArcGIS (Hosted)", description="Title *", layout=W.Layout(width="720px"))
tags       = W.Text(value="connector, edc, dataspace", description="Tags", layout=W.Layout(width="720px"))
desc       = W.Textarea(value="Data pulled via connector and published from ArcGIS Pro.", description="Description", layout=W.Layout(width="720px", height="64px"))

auth_mode  = W.ToggleButtons(options=[("Use Pro login","pro"),("Portal creds","portal")],
                             value="pro", description="ArcGIS Auth *")
portal_url = W.Text(value="https://your-portal/portal", description="Portal URL *", layout=W.Layout(width="720px"))
portal_user= W.Text(value="", description="Username *", layout=W.Layout(width="350px"))
portal_pass= W.Password(value="", description="Password *", layout=W.Layout(width="350px"))

# Buttons + status + log + progress
run_btn    = W.Button(description=" Start", button_style="success", icon="play")
stop_btn   = W.Button(description=" Stop", button_style="danger", icon="stop", disabled=True)
save_btn   = W.Button(description=" Save config", button_style="info", icon="save")
load_btn   = W.Button(description=" Load config", icon="folder-open")

status_msg = W.HTML(value="")
progress   = W.IntProgress(value=0, min=0, max=100, bar_style='', orientation='horizontal', layout=W.Layout(width="100%"))
output     = W.Output(layout=W.Layout(css_classes=["edc-log"]))

# Cards / layout
conn_card = W.VBox([
    W.HTML('<div class="edc-title">Connection & Contract</div>'),
    mode,
    mgmt_url, dsp_url, connector_id,
    W.HBox([api_key, bearer_tok]),
    agreement,
    W.HTML('<div class="edc-muted">Provide either API Key or Bearer (one is enough).</div>'),
], layout=W.Layout(css_classes=["edc-card"]))

pull_card = W.VBox([
    W.HTML('<div class="edc-title">Pull settings</div>'),
    W.HBox([asset_id, http_method]),
    http_path, http_query, http_body,
], layout=W.Layout(css_classes=["edc-card"]))

push_card = W.VBox([
    W.HTML('<div class="edc-title">Push settings</div>'),
    push_base_url, push_note,
], layout=W.Layout(css_classes=["edc-card"]))

out_card = W.VBox([
    W.HTML('<div class="edc-title">Output & ArcGIS</div>'),
    out_file, title, tags, desc,
    auth_mode, portal_url, W.HBox([portal_user, portal_pass]),
], layout=W.Layout(css_classes=["edc-card"]))

buttons_row = W.HBox([run_btn, stop_btn, save_btn, load_btn], layout=W.Layout(css_classes=["edc-buttons"]))

# Mount UI
display(conn_card, W.HBox([pull_card, push_card], layout=W.Layout(gap="12px", width="100%")), out_card, buttons_row, progress, status_msg, output)

# ---------------------------- Dynamic UI rules ----------------------------
def _toggle_visibility(*_):
    is_pull = (mode.value == "pull")
    pull_card.layout.display = "block" if is_pull else "none"
    push_card.layout.display = "block" if not is_pull else "none"

    show_portal = (auth_mode.value == "portal")
    portal_url.layout.display  = "block" if show_portal else "none"
    portal_user.layout.display = "block" if show_portal else "none"
    portal_pass.layout.display = "block" if show_portal else "none"

for w in (mode, auth_mode):
    w.observe(_toggle_visibility, names="value")
_toggle_visibility()

# ---------------------------- Save / Load config ----------------------------
def _gather():
    return {
        "mode": mode.value,
        "mgmt_url": mgmt_url.value.strip(),
        "dsp_url": dsp_url.value.strip(),
        "connector_id": connector_id.value.strip() or None,
        "agreement": agreement.value.strip(),
        "api_key": api_key.value,
        "bearer": bearer_tok.value or None,
        "asset_id": asset_id.value.strip() or None,
        "http_method": http_method.value,
        "http_path": http_path.value.strip() or None,
        "http_query": http_query.value.strip() or None,
        "http_body": http_body.value if http_body.value.strip() else None,
        "push_base_url": push_base_url.value.strip() or None,
        "out_file": out_file.value.strip(),
        "title": title.value.strip(),
        "tags": tags.value.strip(),
        "desc": desc.value,
        "auth_mode": auth_mode.value,
        "portal_url": portal_url.value.strip(),
        "portal_user": portal_user.value.strip(),
        "verify_ssl": True,
        "timeout": 120
    }

def _save(_):
    with open("connector_arcgis_config.json","w",encoding="utf-8") as f:
        json.dump(_gather(), f, indent=2)
    status_msg.value = '<div class="edc-msg-ok">✅ Config saved → connector_arcgis_config.json</div>'

def _load(_):
    p = "connector_arcgis_config.json"
    if not os.path.isfile(p):
        status_msg.value = f'<div class="edc-msg-err">No config file found: {p}</div>'; return
    with open(p,"r",encoding="utf-8") as f:
        cfg = json.load(f)

    mode.value        = cfg.get("mode","pull")
    mgmt_url.value    = cfg.get("mgmt_url", mgmt_url.value)
    dsp_url.value     = cfg.get("dsp_url", dsp_url.value)
    connector_id.value= cfg.get("connector_id","") or ""
    agreement.value   = cfg.get("agreement", agreement.value)
    api_key.value     = cfg.get("api_key","")
    bearer_tok.value  = cfg.get("bearer","") or ""
    asset_id.value    = cfg.get("asset_id","") or ""
    http_method.value = cfg.get("http_method", http_method.value)
    http_path.value   = cfg.get("http_path","") or ""
    http_query.value  = cfg.get("http_query","") or ""
    http_body.value   = cfg.get("http_body","") or ""
    push_base_url.value = cfg.get("push_base_url","") or ""
    out_file.value    = cfg.get("out_file", out_file.value)
    title.value       = cfg.get("title", title.value)
    tags.value        = cfg.get("tags", tags.value)
    desc.value        = cfg.get("desc", desc.value)
    auth_mode.value   = cfg.get("auth_mode", "pro")
    portal_url.value  = cfg.get("portal_url", portal_url.value)
    portal_user.value = cfg.get("portal_user", portal_user.value)
    _toggle_visibility()
    status_msg.value = '<div class="edc-msg-ok">✅ Config loaded.</div>'

save_btn.on_click(_save)
load_btn.on_click(_load)

# ---------------------------- Validation ----------------------------
def _validate(cfg):
    missing = []
    if not cfg["mgmt_url"]: missing.append("Consumer Mgmt API")
    if not cfg["dsp_url"]:  missing.append("Provider DSP")
    if not cfg["agreement"]: missing.append("Contract ID")
    if not (cfg["api_key"] or cfg["bearer"]): missing.append("API Key or Bearer")
    if cfg["mode"]=="pull" and not cfg["out_file"]: missing.append("Save to")
    if cfg["mode"]=="push" and not cfg["push_base_url"]: missing.append("Sink baseUrl")
    if not cfg["title"]:    missing.append("Title")
    if cfg["auth_mode"]=="portal":
        if not portal_url.value.strip():  missing.append("Portal URL")
        if not portal_user.value.strip(): missing.append("Portal Username")
        if not portal_pass.value.strip(): missing.append("Portal Password")
    return missing

# ---------------------------- Run / Stop logic ----------------------------
def _set_running(running: bool):
    run_btn.disabled = running
    stop_btn.disabled = not running
    run_btn.icon = "spinner" if running else "play"
    run_btn.description = " Working…" if running else " Start"
    progress.bar_style = "info" if running else ""
    progress.value = 5 if running else 0

def _stop_clicked(_):
    stop_event.set()
    with output:
        print("⛔ Stop requested by user.")
       
        try:
            cfg = _gather()
            if active_transfer_id["id"]:
                ok = try_terminate_transfer(cfg["mgmt_url"], active_transfer_id["id"], cfg["api_key"], cfg["bearer"], log_out=output)
                print("Terminate requested on connector:", "OK" if ok else "not supported / failed")
        except Exception as ex:
            print("Terminate error:", ex)

def _run_clicked(_):
    status_msg.value=""; output.clear_output()
    stop_event.clear()
    active_transfer_id["id"] = None
    _set_running(True)

    try:
        cfg = _gather()
        missing = _validate(cfg)
        if missing:
            status_msg.value = '<div class="edc-msg-err">Missing required fields: ' + ", ".join(missing) + '</div>'
            return

        with output:
            print("▶ Config (masked):")
            red = dict(cfg)
            if red.get("api_key"): red["api_key"] = "***"
            if red.get("bearer"):  red["bearer"]  = "***"
            print(json.dumps(red, indent=2))

            print("\n⏩ Initiating transfer…")

        tp = start_transfer(
            mgmt_url=cfg["mgmt_url"],
            provider_dsp=cfg["dsp_url"],
            api_key=cfg["api_key"] or None,
            bearer=cfg["bearer"],
            agreement_id=cfg["agreement"],
            transfer_type="HttpData-PULL" if cfg["mode"]=="pull" else "HttpData-PUSH",
            asset_id=cfg["asset_id"],
            http_method=cfg["http_method"],
            http_path=cfg["http_path"],
            http_query=cfg["http_query"],
            http_body=cfg["http_body"],
            push_base_url=cfg["push_base_url"],
            connector_id=cfg["connector_id"],
            verify_ssl=True,
            timeout=cfg["timeout"],
            log_out=output
        )
        progress.value = 20

        if cfg["mode"] == "pull":
            with output: print("\n⏩ Polling until COMPLETED…  (press Stop to cancel)")
            st = poll_until_completed(
                mgmt_url=cfg["mgmt_url"],
                tp_id=tp,
                api_key=cfg["api_key"] or None,
                bearer=cfg["bearer"],
                verify_ssl=True,
                timeout=cfg["timeout"],
                max_polls=120,
                poll_s=2,
                log_out=output
            )
            progress.value = 50

            with output: print("\n⏩ Fetching EDR…")
            edr = fetch_edr(
                mgmt_url=cfg["mgmt_url"],
                tp_id=tp,
                api_key=cfg["api_key"] or None,
                bearer=cfg["bearer"],
                verify_ssl=True,
                timeout=cfg["timeout"],
                retries=8,
                wait_s=1.5,
                log_out=output
            )
            progress.value = 65

            with output: print("\n⏩ Downloading via EDR…")
            saved_path = download_via_edr(
                edr=edr,
                out_path=cfg["out_file"],
                verify_ssl=True,
                timeout=cfg["timeout"],
                retries=4,
                wait_s=1.5,
                log_out=output
            )
            progress.value = 80

            with output: print(f"\n⏩ Publishing to ArcGIS: {saved_path}")
            item = publish_to_arcgis(
                file_path=saved_path,
                title=cfg["title"],
                tags=cfg["tags"],
                desc=cfg["desc"],
                auth_mode=cfg["auth_mode"],
                portal_url=cfg["portal_url"],
                portal_user=cfg["portal_user"],
                portal_pass=portal_pass.value,
                log_out=output
            )
            progress.value = 100
            status_msg.value = '<div class="edc-msg-ok">✅ Pull completed and published.</div>'
            with output: print("✅ Published:", getattr(item, "id", item))
        else:
            progress.value = 60
            with output:
                print("\nℹ PUSH mode: provider will deliver to your sink baseUrl.")
                print("   This UI does not download; monitor your sink logs.")
            progress.value = 100
            status_msg.value = '<div class="edc-msg-ok">✅ Push initiated.</div>'

    except Cancelled as c:
        status_msg.value = f'<div class="edc-msg-err">⛔ {str(c)}</div>'
        with output:
            print("⛔ Cancelled by user.")
    except Exception as ex:
        status_msg.value = f'<div class="edc-msg-err">❌ {str(ex)}</div>'
        with output:
            print("❌ Error:", ex)
            traceback.print_exc()
    finally:
        _set_running(False)

run_btn.on_click(_run_clicked)
stop_btn.on_click(_stop_clicked)
